In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os

In [2]:
from etherscan.accounts import Account

In [3]:
import brownie

brownie.network.connect("mainnet")

In [4]:
import requests as requests

In [5]:
block_info_response = requests.get("https://api.blockcypher.com/v1/eth/main")
current_block = block_info_response.json()['height']

In [6]:
crv_depositor_contract = "0x8014595F2AB54cD7c604B00E9fb932176fDc86Ae"
crv_depositor_contract_obj = brownie.network.contract.Contract.from_explorer(crv_depositor_contract)

Fetching source of 0x8014595F2AB54cD7c604B00E9fb932176fDc86Ae from api.etherscan.io...


/Users/swadhinnanda/Projects/git/on-chain-analytics/venv/lib/python3.7/site-packages/brownie/network/contract.py:1239: BrownieCompilerWarning: 0x8014595F2AB54cD7c604B00E9fb932176fDc86Ae: Locally compiled and on-chain bytecode do not match!
  BrownieCompilerWarning,


In [7]:
go_back = 100000
query = f"https://api.etherscan.io/api?module=account&action=txlist&address={crv_depositor_contract}&startblock={current_block-go_back}&endblock={current_block}&sort=asc&page=1&offset=10000&apikey={os.environ['ETHERSCAN_API_KEY']}"
etherscan_api_response = requests.post(query)
etherscan_api_response

<Response [200]>

In [8]:
crv_depositor_txes = etherscan_api_response.json()['result']
len(crv_depositor_txes)

849

In [9]:
example_tx = crv_depositor_txes[10]
tx_receipt = brownie.network.transaction.TransactionReceipt(example_tx['hash'])
tx_receipt.info()

Transaction was Mined 
---------------------
Tx Hash: 0xd9df0e55888b54bfe697a788c34aebee2b6beab906d049e9ee7eacdc13455ed7
From: 0x6a5282a527E41a117F4a46A7fc46AB09EDF69088
To: 0x8014595F2AB54cD7c604B00E9fb932176fDc86Ae
Value: 0
Function: CrvDepositor.deposit
Block: 13280695
Gas Used: 476206 / 881020 (54.1%)

Events In This Transaction
--------------------------
├── Curve DAO Token (0xD533a949740bb3306d119CC777fa900bA034cd52)
│   ├── Transfer
│   │   ├── _from: 0x6a5282a527E41a117F4a46A7fc46AB09EDF69088
│   │   ├── _to: 0x989AEb4d175e16225E39E87d0D97A3360524AD80
│   │   └── _value: 3256797609518248424466
│   ├── Transfer
│   │   ├── _from: 0x8014595F2AB54cD7c604B00E9fb932176fDc86Ae
│   │   ├── _to: 0x989AEb4d175e16225E39E87d0D97A3360524AD80
│   │   └── _value: 24955793121825824798054
│   ├── Approval
│   │   ├── _owner: 0x989AEb4d175e16225E39E87d0D97A3360524AD80
│   │   ├── _spender: 0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2
│   │   └── _value: 0
│   ├── Approval
│   │   ├── _owner: 0x98

In [ ]:
crv_lock_txes = {'timestamp': [], 'qt': [], 'gas_price': []}
for tx in crv_depositor_txes:
    try:
        tx_receipt = brownie.network.transaction.TransactionReceipt(tx['hash'])
        if tx_receipt.fn_name == 'deposit':
            crv_lock_txes['qt'].append(tx_receipt.events[0]['_value'] * 1e-18)
            crv_lock_txes['gas_price'].append(tx_receipt.gas_price * 1e-9)
            crv_lock_txes['timestamp'].append(datetime.datetime.fromtimestamp(tx_receipt.timestamp))
    except:
        continue

In [ ]:
df_crv_lock_txes = pd.DataFrame(crv_lock_txes)
df_crv_lock_txes.set_index('timestamp', inplace=True)
df_crv_lock_txes

In [ ]:
df_crv_lock_daily = pd.DataFrame()
df_crv_lock_daily['avg_gas'] = df_crv_lock_txes.groupby(pd.Grouper(freq='1D'))['gas_price'].mean()
df_crv_lock_daily['total_locked'] = df_crv_lock_txes.groupby(pd.Grouper(freq='1D'))['qt'].sum() / 10000
df_crv_lock_daily

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
df_crv_lock_daily.plot(ax=ax, kind='bar')
ax.set_title("total locked crv (x10000) and average gas each day")